In [1]:
# Setup
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
pd.set_option('mode.chained_assignment', 'raise')
pl23 = pd.read_csv('Final.csv')
pl23 = pl23.set_index('Team')
print(pl23)

                   Rank  Unnamed: 2  MP  Won  Drawn  Lost  GF  GA  GD  Points
Team                                                                         
Man City              1         NaN  38   28      5     5  94  33  61      89
Arsenal               2         NaN  38   26      6     6  88  43  45      84
Man Utd               3         NaN  38   23      6     9  58  43  15      75
Newcastle             4         NaN  38   19     14     5  68  33  35      71
Liverpool             5         NaN  38   19     10     9  75  47  28      67
Brighton              6         NaN  38   18      8    12  72  53  19      62
Aston Villa           7         NaN  38   18      7    13  51  46   5      61
Spurs                 8         NaN  38   18      6    14  70  63   7      60
Brentford             9         NaN  38   15     14     9  58  46  12      59
Fulham               10         NaN  38   15      7    16  55  53   2      52
Crystal Palace       11         NaN  38   11     12    15  40  4

In [8]:
def fetch_coordinates(df, dim):
    coordinates = []
    for i in range(len(df)):
        for j in range(dim):
            coordinates.append(df.iloc[i,j])
    return coordinates

In [3]:
def calculate_distance_assign_clusters(df, centroids, xaxis, yaxis):
    c0x, c0y, c1x, c1y, c2x, c2y, c3x, c3y = fetch_coordinates(centroids, 2)
    df['dist_centroid_0'] = df.apply(calculate_distance, args=(c0x, c0y, xaxis, yaxis), axis=1)
    df['dist_centroid_1'] = df.apply(calculate_distance, args=(c1x, c1y, xaxis, yaxis), axis=1)
    df['dist_centroid_2'] = df.apply(calculate_distance, args=(c2x, c2y, xaxis, yaxis), axis=1)
    df['dist_centroid_3'] = df.apply(calculate_distance, args=(c3x, c3y, xaxis, yaxis), axis=1)
    
    mask_0 = (df['dist_centroid_0'] < df['dist_centroid_2']) & (df['dist_centroid_0'] < df['dist_centroid_1']) & (df['dist_centroid_0'] < df['dist_centroid_3'])
    mask_1 = (df['dist_centroid_1'] < df['dist_centroid_2']) & (df['dist_centroid_1'] < df['dist_centroid_0']) & (df['dist_centroid_1'] < df['dist_centroid_3'])
    mask_2 = (df['dist_centroid_2'] < df['dist_centroid_0']) & (df['dist_centroid_2'] < df['dist_centroid_1']) & (df['dist_centroid_2'] < df['dist_centroid_3'])
    mask_3 = (df['dist_centroid_3'] < df['dist_centroid_2']) & (df['dist_centroid_3'] < df['dist_centroid_1']) & (df['dist_centroid_3'] < df['dist_centroid_0'])
    df.loc[mask_0, 'cluster'] = 0
    df.loc[mask_1, 'cluster'] = 1
    df.loc[mask_2, 'cluster'] = 2
    df.loc[mask_3, 'cluster'] = 3
    
    return df

In [4]:
def calculate_distance(s, cx, cy, xaxis, yaxis):
    distance = np.sqrt((s.loc[xaxis] - cx)**2 + (s.loc[yaxis] - cy)**2)
    return distance

In [5]:
# Define a function that takes in the values you want to use to measure
def kmeans(df, xaxis, yaxis, n, loops):
    df = df[[xaxis, yaxis]].copy()
    df['cluster'] = 0
    centroids = df.sample(n)
    print(centroids)
    for i in range(loops):
        df = calculate_distance_assign_clusters(df, centroids, xaxis, yaxis)
        centroids = df.groupby(['cluster'])[[xaxis, yaxis]].mean().reset_index()
        centroids.drop('cluster', axis=1, inplace=True)
    sns.scatterplot(data=centroids, x=xaxis, y=yaxis, color='black', s=150)
    sns.scatterplot(data=df, x=xaxis, y=yaxis, hue='cluster', palette='tab10')
    return [df, centroids]

In [6]:
kmeans(pl23, 'Lost', 'GA', 4, 10000)

             Lost  GA  cluster
Team                          
Aston Villa    13  46        0
Arsenal         6  43        0
Brentford       9  46        0
Bournemouth    21  71        0


ValueError: not enough values to unpack (expected 8, got 4)